***
***


# <center>Validação Cruzada</center>

<center><img src="https://pessoalex.files.wordpress.com/2019/04/validacaocruzada.gif"></center>

***
***

**Bibliotecas Utilizadas**

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split as tts

**Importando base de dados**

In [2]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


**Separando os dados de treino e teste**

In [3]:
# Fixando um seed
np.random.seed(2020)

# Separando todos os dados em variáveis genéricas x e y
x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

# Utilizando o train_test_split da biblioteca do scikit-learning
train_x, test_x, train_y, test_y = tts(x, y, test_size=.2, stratify=y)

**Criando um Algoritmo de Base - Baseline**

In [4]:
from sklearn.dummy import DummyClassifier

In [5]:
# Instânciando o DummyClassifier
dummy = DummyClassifier(strategy='stratified', random_state=2020)

# Visualizando o estimador
dummy

DummyClassifier(constant=None, random_state=2020, strategy='stratified')

In [6]:
# Treinando o estimador
dummy.fit(train_x, train_y)

DummyClassifier(constant=None, random_state=2020, strategy='stratified')

In [7]:
# Visualizando a Acurácia do Modelo / Taxa de acerto
print(f'Accuracy-: {(dummy.score(test_x, test_y) * 100):.2f}%')

Accuracy-: 50.15%


### Modelo utilizando arvore de decisão

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [9]:
# Instânciando um DecisionTreeClassifer
modelo = DecisionTreeClassifier(max_depth=5, random_state=2020)

# Visualizando o modelo
modelo

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=2020, splitter='best')

In [10]:
# Treinando o DecisionTreeClassifier
modelo.fit(train_x, train_y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=2020, splitter='best')

In [11]:
# Gerando previsões com o modelo
previsoes = modelo.predict(test_x)

# Accuracy
print(f'Accuracy-: {(accuracy_score(test_y, previsoes) * 100):.2f}%')

Accuracy-: 76.95%


## Validação Cruzada

In [12]:
from sklearn.model_selection import cross_validate

**Modelo que será validado**

In [13]:
# Instânciando um DecisionTreeClassifer
modelo = DecisionTreeClassifier(max_depth=5, random_state=2020)

# Visualizando o modelo
modelo

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=2020, splitter='best')

**Validação Simples**

In [14]:
# Definindo um função para o relatório de validação
def relatorio(results):
    
    media = results['test_score'].mean()
    std = results['test_score'].std()
    
    print('--' * 21)
    print(f' Intervalo de Confiança-: [{(media - 2 * std) * 100:.2f}%, {(media + 2 * std) * 100:.2f}%]')
    print(f' Média das Accuracy-: {(media * 100):.2f}%')
    print('--' * 21)

In [15]:
results = cross_validate(modelo, x, y, cv=5)

In [16]:
relatorio(results)

------------------------------------------
 Intervalo de Confiança-: [77.42%, 79.60%]
 Média das Accuracy-: 78.51%
------------------------------------------


**Validação Cruzada com KFold, shuffle = True**

In [17]:
from sklearn.model_selection import KFold

In [18]:
# Definindo o KFold
cv = KFold(n_splits=10, shuffle=True, random_state=2020)

# VALIDAÇÃO CRUZADA
results = cross_validate(modelo, x, y, cv=cv)

In [19]:
# Relatório de validação cruzada
relatorio(results)

------------------------------------------
 Intervalo de Confiança-: [77.33%, 80.13%]
 Média das Accuracy-: 78.73%
------------------------------------------


## Validação Cruzada - StratifiedKFold

In [20]:
from sklearn.model_selection import StratifiedKFold

In [21]:
# Colocando os dados em forma crescente em relação a coluna vendido
dados_azar = dados.sort_values('vendido', ascending = True)

x_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']

dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


**Realizando a validação cruzada com KFold**

In [22]:
results = cross_validate(modelo, x_azar, y_azar, cv=cv)

In [23]:
relatorio(results)

------------------------------------------
 Intervalo de Confiança-: [75.73%, 81.03%]
 Média das Accuracy-: 78.38%
------------------------------------------


**Realizando a validação com StratifiedKFold**

In [24]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=2020)

results = cross_validate(modelo, x_azar, y_azar, cv=cv)

In [25]:
relatorio(results)

------------------------------------------
 Intervalo de Confiança-: [75.31%, 81.63%]
 Média das Accuracy-: 78.47%
------------------------------------------


## Validação por Grupos - GroupKFold

In [26]:
from sklearn.model_selection import GroupKFold

In [27]:
dados['Modelo'] = dados['idade_do_modelo'] + np.random.randint(-2, 3, dados.shape[0]) + 2

**Realizando a validação cruzada com GroupKFold**

> A validação cruzada com <b>GroupKFold</b> nos permite como vai ser o desenpenho do nosso modelo para novos modelos de carros que ele ainda não conhece.


In [28]:
cv = GroupKFold(n_splits=10)

results = cross_validate(modelo, x_azar, y_azar, groups=dados['Modelo'], cv=cv)

In [29]:
# Visualizando o relatório da validação cruzada com o GroupKFold
relatorio(results)

------------------------------------------
 Intervalo de Confiança-: [75.37%, 81.86%]
 Média das Accuracy-: 78.61%
------------------------------------------


## Validação cruzada com Pipeline - SVC

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [31]:
# Instânciando o SVC e o StandardScaler
svc = SVC()
scaler = StandardScaler()

# Visualizando o SVC
svc

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [32]:
# Criando um Pipeline
pipeline = Pipeline([('Scaler', scaler), ('Modelo', svc)])

# Visualizando o pipeline
pipeline

Pipeline(memory=None,
         steps=[('Scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('Modelo',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [33]:
# Realizando o cross_validade com Pipeline
results = cross_validate(pipeline, x_azar, y_azar, groups=dados['Modelo'], cv=cv)

In [34]:
# Visualizando resultados da validação
relatorio(results)

------------------------------------------
 Intervalo de Confiança-: [73.72%, 80.01%]
 Média das Accuracy-: 76.86%
------------------------------------------
